In [12]:
import pandas as pd
import pickle
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
from statsmodels.iolib.smpickle import load_pickle

dependent_vars = ['Depressed', 'Anxious', 'Received_Free_Food', 'Food_Insecurity', 'Difficulty_with_Expenses', 'Rent_Confidence']

In [13]:

results = []
for var in dependent_vars:
    models = []
    for kids in ["None", "1", "2", "3", "4"]:
        models.append(load_pickle(f"models\{var}_fe_model_{kids}_kids.p"))
    output_table = pd.read_html(
        summary_col([*models],
                    stars=True,
                    regressor_order=["Treat"],
                    drop_omitted=True,
                    info_dict={
            "N": lambda x: (str(int(x.nobs))), "Fixed Effects": lambda x: "Week/Group"})
        .as_html(), header=0, index_col=0)[0]
    output_table.columns = ['Children Generally', 'One Child', 'Two Children', 'Three Children', 'Four Children']
    output_table.index = 'Treat', '', 'R-squared', 'R-squared Adj.', 'N', 'Fixed Effects'
    results.append(output_table)

KeyboardInterrupt: 

In [ ]:
new_index = []
for ind, var in enumerate(dependent_vars):
    new_index.append(var)
    if ind+1 % 2 != 0:
        new_index.append("")

In [ ]:
all_results = pd.concat([result.iloc[:2, :] for result in results])
all_results.index = new_index
print(pd.concat([all_results, results[0].iloc[-2:, :]]).to_csv().replace("_", " "))

,Children Generally,One Child,Two Children,Three Children,Four Children
Depressed,0.0002,0.0030,-0.0050,0.0081,-0.0056
,(0.0043),(0.0057),(0.0065),(0.0113),(0.0172)
Anxious,0.0176***,0.0169***,0.0115,0.0210*,0.0577***
,(0.0047),(0.0064),(0.0073),(0.0125),(0.0209)
Received Free Food,-0.0181***,-0.0141***,-0.0168***,-0.0218**,-0.0485***
,(0.0033),(0.0042),(0.0053),(0.0095),(0.0169)
Food Insecurity,-0.0256***,-0.0206***,-0.0264***,-0.0375***,-0.0395**
,(0.0037),(0.0049),(0.0058),(0.0090),(0.0180)
Difficulty with Expenses,-0.0052,-0.0031,-0.0111,-0.0024,-0.0006
,(0.0051),(0.0068),(0.0080),(0.0133),(0.0215)
Rent Confidence,0.0127**,0.0099,0.0184**,0.0155,0.0286
,(0.0051),(0.0068),(0.0080),(0.0132),(0.0216)
N,842783,713637,679113,628867,608418
Fixed Effects,Week/Group,Week/Group,Week/Group,Week/Group,Week/Group



In [16]:

models = []
for var in dependent_vars:
    models.append(load_pickle(f"models\{var}_event_study_None_kids.p"))


In [21]:
output_table = pd.read_html(
    summary_col([*models],
                stars=True,
                info_dict={
        "N": lambda x: (str(int(x.nobs)))})
    .as_html(), header=0, index_col=0)[0]
# output_table.columns = ['Children Generally', 'One Child',
#                         'Two Children', 'Three Children', 'Four Children']
# output_table.index = 'Treat', '', 'R-squared', 'R-squared Adj.', 'N', 'Fixed Effects'
# results.append(output_table)

In [24]:
output_table = output_table.reset_index()

In [37]:
print(output_table[output_table['index'].str.startswith("C(Week)") & output_table['index'].str.endswith(":Eligible")].rename(columns={"index": "Coefficient"}).to_csv(index=False).replace("_", " "))

Coefficient,Depressed,Anxious,Received Free Food,Food Insecurity,Difficulty with Expenses,Rent Confidence
C(Week)[T.27]:Eligible,-0.0188,-0.0081,-0.0204**,-0.0231**,-0.0701***,0.0284**
C(Week)[T.28]:Eligible,-0.0106,0.0083,-0.0113,-0.0175,-0.0063,0.0284*
C(Week)[T.29]:Eligible,-0.0003,-0.0065,-0.0194**,-0.0101,-0.0308**,0.0291**
C(Week)[T.30]:Eligible,-0.0028,0.0128,-0.0076,-0.0007,-0.0117,0.0112
C(Week)[T.31]:Eligible,-0.0024,0.0108,-0.0268***,-0.0072,-0.0016,0.0081
C(Week)[T.32]:Eligible,0.0046,0.0101,-0.0289***,-0.0008,-0.0215,0.0182
C(Week)[T.33]:Eligible,0.0054,0.0284**,-0.0169*,0.0087,0.0029,0.0251*
C(Week)[T.34]:Eligible,0.0045,0.0374***,-0.0425***,-0.0358***,-0.0182,0.0056
C(Week)[T.35]:Eligible,-0.0007,0.0307**,-0.0582***,-0.0416***,-0.0133,0.0459***
C(Week)[T.36]:Eligible,0.0003,0.0429***,-0.0522***,-0.0261**,-0.0311**,0.0221
C(Week)[T.37]:Eligible,-0.0092,0.0118,-0.0536***,-0.0347***,-0.0163,0.0216
C(Week)[T.38]:Eligible,-0.0036,0.0194,-0.0539***,-0.0324***,-0.0120,0.0294**


In [28]:
output_table

,index,Depressed,Anxious,Received_Free_Food,Food_Insecurity,Difficulty_with_Expenses,Rent_Confidence
0,Intercept,0.2177***,0.3005***,0.0595***,0.0879***,0.3059***,0.5891***
1,NaN,(0.0044),(0.0048),(0.0028),(0.0038),(0.0051),(0.0051)
2,C(Week)[T.27],-0.0060,-0.0167**,-0.0021,-0.0056,-0.0348***,0.0146**
3,NaN,(0.0064),(0.0068),(0.0038),(0.0054),(0.0072),(0.0074)
4,C(Week)[T.28],-0.0266***,-0.0413***,-0.0059,-0.0192***,-0.0784***,0.0006
...,...,...,...,...,...,...,...
78,C(Week)[T.45]:Eligible,0.0100,0.0261*,-0.0460***,0.0035,0.0074,-0.0016
79,NaN,(0.0130),(0.0138),(0.0103),(0.0119),(0.0146),(0.0147)
80,R-squared,0.0014,0.0058,0.0154,0.0088,0.0255,0.0012
81,R-squared Adj.,0.0013,0.0058,0.0153,0.0088,0.0255,0.0012
